In [27]:
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split

import json
import numpy as np
import pandas as pd
import tensorflow as tf
import os

In [18]:
with open('json/string_test.json', encoding='UTF8') as file:
    dataset = json.load(file)

In [19]:
print(dataset[1])

{'지역': '대구', '계열': '자연', '대학명': 'DGIST', '학과명': '융복합대학(기초학부)', '전형유형': '종합', '전형명': '학교장추천', '최저적용유무': '', '최초지원결과': '합', '최종지원결과': '합', '전과목(평균)': '1.47', '국영수사(평균)': '1.51', '국영수과(평균)': '1.29', '국영수탐(백분위)': '281.5', '국영수탐(등급)': '1.67'}


In [20]:
# JSON 데이터를 pandas DataFrame으로 변환
df = pd.DataFrame(dataset)

# "최저 지원 유무" 열 삭제
df_modify = df.drop(columns=['최저적용유무', '최초지원결과'])

# 빈 값이 있는 행 제거
df_modify = df_modify.replace('', pd.NA).dropna()

# 결과를 다시 JSON 형식으로 변환
modified_data = df_modify.to_dict(orient='records')

In [21]:
print(modified_data[1])

{'지역': '대구', '계열': '자연', '대학명': 'DGIST', '학과명': '융복합대학(기초학부)', '전형유형': '종합', '전형명': '학교장추천', '최종지원결과': '합', '전과목(평균)': '1.47', '국영수사(평균)': '1.51', '국영수과(평균)': '1.29', '국영수탐(백분위)': '281.5', '국영수탐(등급)': '1.67'}


In [22]:
# 숫자로 변환하여 NumPy 배열 생성
numeric_data = np.array([
    [
        data['전과목(평균)'],
        data['국영수사(평균)'],
        data['국영수과(평균)'],
        data['국영수탐(백분위)'],
        data['국영수탐(등급)']
    ] 
    for data in modified_data
])

# 문자열 데이터를 numpy 배열로 변환 (hashing trick 적용) --> 수정 필요
string_data = np.array([
    [
        hash(data['지역']),
        hash(data['계열']),
        hash(data['대학명']),
        hash(data['학과명']),
        hash(data['전형유형']),
        hash(data['전형명'])
    ]
    for data in modified_data
])

# 결과값을 정수로 매핑 ("최종지원결과"를 1 또는 0으로 변환)
result = np.array([data['최종지원결과'] for data in modified_data])

In [23]:
print(numeric_data[0:5])
print(result[0:5])

[['1.32' '1.31' '1.22' '252.5' '2.67']
 ['1.47' '1.51' '1.29' '281.5' '1.67']
 ['1.3' '1.3' '1.3' '225.5' '3.67']
 ['1.39' '1.44' '1.33' '224' '3.33']
 ['1.59' '1.41' '1.36' '263.5' '2.17']]
['합' '합' '합' '합' '불']


In [24]:
# 2차원 배열의 각 행에 대해 문자열을 정수로 변환
X = np.array(numeric_data).astype(float)

# y값 문자열에 대해 원-핫 인코딩
y = np.array(result)
e = LabelEncoder()
e.fit(y)
Y = e.transform(y)

print(X[0:5])
print(Y[0:5])


[[  1.32   1.31   1.22 252.5    2.67]
 [  1.47   1.51   1.29 281.5    1.67]
 [  1.3    1.3    1.3  225.5    3.67]
 [  1.39   1.44   1.33 224.     3.33]
 [  1.59   1.41   1.36 263.5    2.17]]
[1 1 1 1 0]


In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, shuffle=True)

In [31]:
model = tf.keras.models.Sequential([
    # 숫자 같은 경우 계속 돌려 보면서 정확도가 높은 방향으로 모델을 튜닝시켜 주는 작업이 필요함
    tf.keras.layers.Dense(24, input_dim=5, activation='tanh'),
    tf.keras.layers.Dense(10, activation='tanh'),
    # 마지막 레이어는 예측결과(sigmoid -> 0~1확률)
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 학습 데이터, 실제 정답
model.fit(np.array(X_train), np.array(Y_train), epochs=100)

score = model.evaluate(X_test, Y_test)
print("Test Accuracy: ", score[1])

Epoch 1/100
2/2 [==============================] - 0s 3ms/step - loss: 0.5734 - accuracy: 0.9286
Epoch 2/100
2/2 [==============================] - 0s 3ms/step - loss: 0.5244 - accuracy: 0.9286
Epoch 3/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4896 - accuracy: 0.9286
Epoch 4/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4570 - accuracy: 0.9286
Epoch 5/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4320 - accuracy: 0.9286
Epoch 6/100
2/2 [==============================] - 0s 2ms/step - loss: 0.4111 - accuracy: 0.9286
Epoch 7/100
2/2 [==============================] - 0s 4ms/step - loss: 0.3927 - accuracy: 0.9286
Epoch 8/100
2/2 [==============================] - 0s 3ms/step - loss: 0.3737 - accuracy: 0.9286
Epoch 9/100
2/2 [==============================] - 0s 2ms/step - loss: 0.3590 - accuracy: 0.9286
Epoch 10/100
2/2 [==============================] - 0s 4ms/step - loss: 0.3465 - accuracy: 0.9286
Epoch 11/100
2/2 [===========

In [10]:
# 모델 생성
model = Sequential()
model.add(Dense(64, activation='tanh'))
model.add(Dense(128, activation='tanh')) 
# 마지막 레이어는 예측결과(sigmoid -> 0~1확률)
model.add(Dense(1, activation='sigmoid')) 

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 실행
model.fit(np.array(X), np.array(Y), epochs=100)

Epoch 1/100
2/2 [==============================] - 0s 3ms/step - loss: 0.5435 - accuracy: 0.8333
Epoch 2/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4536 - accuracy: 0.8333
Epoch 3/100
2/2 [==============================] - 0s 2ms/step - loss: 0.5013 - accuracy: 0.8333
Epoch 4/100
2/2 [==============================] - 0s 2ms/step - loss: 0.5009 - accuracy: 0.8333
Epoch 5/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4721 - accuracy: 0.8333
Epoch 6/100
2/2 [==============================] - 0s 2ms/step - loss: 0.4482 - accuracy: 0.8333
Epoch 7/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4459 - accuracy: 0.8333
Epoch 8/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4595 - accuracy: 0.8333
Epoch 9/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4539 - accuracy: 0.8333
Epoch 10/100
2/2 [==============================] - 0s 3ms/step - loss: 0.4435 - accuracy: 0.8333
Epoch 11/100
2/2 [===========

In [26]:
# 예측
predict = model.predict([ [1.32, 1.31, 1.22, 252.5, 2.67], [5, 4.5, 4.8, 72, 6.2] ])
print(predict)

1/1 [==============================] - 0s 62ms/step
[[0.7141056 ]
 [0.00170209]]


In [12]:
# 모델 생성
model = Sequential()
model.add(Dense(64, activation='tanh'))
model.add(Dense(128, activation='tanh')) 
# 마지막 레이어는 예측결과(sigmoid -> 0~1확률)
model.add(Dense(1, activation='sigmoid')) 

# 모델 컴파일
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 모델 저장 폴더 만들기
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)
    
modelpath = "./model/best_model.h5"

# 모델 업데이트 및 저장(앞에 모델보다 나아졌을때만 저장)
checkpointer = ModelCheckpoint(filepath=modelpath, moniter='loss', verbos=1, save_best_only=True)

# 학습 자동 중단 설정(학습을 계속해도 정확도에 진전이 없다면 stop)
early_stopping_callback = EarlyStopping(monitor='loss', patience=100)

# 모델 실행
model.fit(np.array(X), np.array(Y), epochs=100, batch_size=32, validation_data=(np.array(X), np.array(Y)), verbose=0, callbacks=[early_stopping_callback, checkpointer])

c:\Users\강민지\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
